In [1]:
import deco
import deco_mix
import deco_helper
import numpy
import untangle
import copy
import matplotlib

# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.
%matplotlib inline

In [2]:
# Globals
settings = deco.deco_settings(
    last_stop = 9,
    gf_low = 0.1,
    gf_high = 0.90)
precision = numpy.float32
profile_time_factor = 1.0
sac_bottom = 18
sac_deco = 15
xml_file = "profiles/2015_12_27T15_42_48.uddf"

In [3]:
doc = untangle.parse(xml_file)

SAXParseException: <unknown>:1:8: not well-formed (invalid token)

In [ ]:
mix_ctx = deco_mix.deco_mix_ctx()

gasmixes = dict()

imix = 0
for mix in doc.uddf.gasdefinitions.mix:
    try:
        switchdepth = int(mix.switchdepth.cdata)
    except:
        switchdepth = 0
    gasmixes.update({ mix['id']: {
        'imix': imix,
        'f_he': float(mix.he.cdata),
        'f_o2': float(mix.o2.cdata),
        'f_n2': 1 - float(mix.o2.cdata) - float(mix.he.cdata) }})
    mix_ctx.set_mix(
        imix,
        deco_mix.deco_mix(f_o2 = float(mix.o2.cdata),
        f_he = float(mix.he.cdata),
        switch_depth = switchdepth,
        enabled = True))
    imix += 1

#print(gasmixes)
#print(mix_ctx)

In [ ]:
# Initialize decompression context
try:
    p_surface_in = float(doc.uddf.profiledata.repetitiongroup.dive.informationbeforedive.surfacepressure.cdata) / 100000
except:
    p_surface_in = 1.0

deco_ctx = deco.deco_zhl16b(
    settings = settings,
    mix_ctx = mix_ctx,
    p_surface = p_surface_in,
    salinity = 1.0)

#print(deco_ctx)

In [ ]:
dive_profile = deco_helper.profile(
        runtime = 0,
        ctx = deco_ctx,
        maxentries = len(doc.uddf.profiledata.repetitiongroup.dive.samples.waypoint))

# Read profile
runtime = 0
depth = 0
lastruntime = 0
lastdepth = 0
for waypoint in doc.uddf.profiledata.repetitiongroup.dive.samples.waypoint:
    try:
        activemix = waypoint.switchmix['ref']
        # eventually switch mix
        dive_profile.ctx.mix_ctx.set_active_mix(gasmixes[activemix]['imix'])
        #print("switch mix: %s" % (dive_profile.ctx.mix_ctx.get_mix(dive_profile.ctx.mix_ctx.get_active_mix())))
    except:
        pass
    
    runtime = float(waypoint.divetime.cdata) / 60.0 * profile_time_factor
    depth = float(waypoint.depth.cdata)

    #print("runtime: %.1f min. depth: %.1f m lastdepth %.1f m" % (runtime, depth, lastdepth))
    
    if (depth != 0 and runtime - lastruntime > 0):
        #%lprun -s -m deco dive_profile.update(time = runtime - lastruntime, depth_begin = lastdepth, depth_end = depth, sac = sac_bottom)
        dive_profile.update(
            time = runtime - lastruntime,
            depth_begin = lastdepth,
            depth_end = depth,
            sac = sac_bottom)
                
    lastruntime = runtime
    lastdepth = depth

# Strip data array to number of actually valid records
dive_profile.strip()

print(dive_profile.di)
print(dive_profile.ctx.mix_ctx)

In [ ]:
data = dive_profile.data

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize = (18, 12))

matplotlib.pyplot.title(
    'ZHL16B-GF {:d}/{:d}'.format(int(deco_ctx.settings.gf_low * 100), int(deco_ctx.settings.gf_high * 100)))

matplotlib.pyplot.xlabel('Time in min.')

matplotlib.pyplot.ylabel('Pressure in mfw\nGF')

ax.plot(
    data['time'],
    data['depth'],
    label = 'Depth in m',
    color = 'Blue',
    lw = 1)

#ax.set_color_cycle([matplotlib.pyplot.cm.Oranges_r(i) for i in numpy.linspace(0, 0.8, 16)])
#ax.plot(
#    data['time'],
#    data['p_tissues_n2'] * 10,
#    lw = 0.3)

#ax.set_color_cycle([matplotlib.pyplot.cm.Greens_r(i) for i in numpy.linspace(0, 0.8, 16)])
#ax.plot(
#    data['time'],
#    data['p_tissues_he'] * 10,
#    lw = 0.3)

ax.plot(
    data['time'],
    data['p_ceiling'] * 10 - 10,
    label = 'Decompression Ceiling in m',
    color = 'Red',
    lw = 2)

ax.plot(
    data['time'],
    data['gf_allowed'] * 100,
    label = 'Permitted GF',
    color = 'Purple',
    lw = 2)

ax.plot(
    data['time'],
    data['gf_current'] * 100,
    label = 'Current GF',
    color = 'Purple',
    lw = 1)

legend = ax.legend(loc = 'upper right', shadow = True)

ax.grid()
ax.invert_yaxis()
